# Task list

In [4]:
import requests
import pandas as pd
%run ../employee/links.ipynb

### All task list - Bio data

In [5]:
def compute_date_progression(df, startDate="startDate", endDate="endDate"):
    df[startDate] = pd.to_datetime(df[startDate])
    df[endDate] = pd.to_datetime(df[endDate])
    # Calculate the total duration of each project in days
    df['duration'] = (df[endDate] - df[startDate]).dt.days
    # Calculate the current progress of each project in days
    today = pd.Timestamp.now().date()
    today = pd.Timestamp(today) # Convert today to pd.Timestamp object
    df['currentProgress'] = (today - df[startDate]).dt.days
    # Calculate the progression as a percentage
    df['progression'] = (df['currentProgress'] / df['duration']) * 100
    df['progression'] = df['progression'].clip(lower=0)
    # Set progression to 100 if it's greater than 100
    df['progression'] = df['progression'].clip(upper=100)
    df['progression'] = df['progression'].round(2)
    df.drop(columns=["duration","currentProgress"], inplace=True)
    return df['progression']

In [12]:
req = requests.get(links_ns.task.all).json() # Risk of empty list
task = pd.json_normalize(req)
task = task[["taskId","code","name","type","comment","startDate","endDate","status"]].copy()
task["progression"] = compute_date_progression(task)
task

,taskId,code,name,type,startDate,endDate,status,progress,budget,comment,...,project.code,project.name,project.type,project.startDate,project.endDate,project.status,project.progress,project.budget,project.comment,project.project
0,1,A-06-21,Audit Diagnostic des CVT,OS,2022-01-06,2022-06-22,FINISHED,100.0,30000000.0,DCSTI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,A-01-22,Rédaction du plan stratégique de l'ARTI,PROJECT,2022-11-17,2023-04-11,IN_PROGRESS,78.0,15000000.0,DSESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,A-02-22,Etude sur la tarification des service de trans...,OS,2022-06-22,2023-05-24,IN_PROGRESS,64.0,100000000.0,DSESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,A-01-23,Application de suivi-évaluation des activités ...,PROJECT,2023-01-02,2023-05-31,IN_PROGRESS,52.0,700000.0,DSESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,A-02-23,Mise en place de l'OTC-CI,PROJECT,2023-06-01,2024-06-30,SCHEDULED,0.0,400000000.0,DSESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,A-07-21,Rédaction des TDR de l'audit diagnostic,ACTIVITY,2022-01-10,2022-01-15,FINISHED,100.0,NaN,DSE,...,A-06-21,Audit Diagnostic des CVT,OS,2022-01-06,2022-06-22,FINISHED,100.0,30000000.0,DCSTI,NaN
6,7,A-08-21,Audit des centres,ACTIVITY,2022-06-06,2022-06-22,FINISHED,100.0,NaN,DCSTI,...,A-06-21,Audit Diagnostic des CVT,OS,2022-01-06,2022-06-22,FINISHED,100.0,30000000.0,DCSTI,NaN
7,8,A-03-23,Prototypage de l'application,ACTIVITY,2023-01-03,2023-02-22,FINISHED,NaN,NaN,DSESP,...,A-01-23,Application de suivi-évaluation des activités ...,PROJECT,2023-01-02,2023-05-31,IN_PROGRESS,52.0,700000.0,DSESP,NaN
8,9,A-04-23,Design des bases de données de l'application,ACTIVITY,2023-03-20,2023-03-24,IN_PROGRESS,NaN,NaN,DSESP,...,A-01-23,Application de suivi-évaluation des activités ...,PROJECT,2023-01-02,2023-05-31,IN_PROGRESS,52.0,700000.0,DSESP,NaN
9,10,A-04-23,Mise en production,ACTIVITY,2023-04-07,2023-04-15,SCHEDULED,NaN,NaN,DSESP,...,A-01-23,Application de suivi-évaluation des activités ...,PROJECT,2023-01-02,2023-05-31,IN_PROGRESS,52.0,700000.0,DSESP,NaN


### Compute ressources and progression

In [7]:
req = requests.get(links_ns.taskrole.all).json() # Risk of empty list
projects = pd.json_normalize(req)

#select_cols = ["taskroleId","role","task.taskId","task.project.taskId","task.code","task.name","task.type",
#               "task.startDate","task.endDate","task.status","position.subdivision.direction.shortName",
#               "position.service.subdivision.direction.shortName","task.comment"]
#rename_cols = ["taskroleId","role","taskId","projectId","code","name","type","start","end","status","dirName","dirName2","dirOwner"]
#select_cols = ["task_id","code","name","type","startDate","endDate","status","progress","project.projectId"]
#select_bio_cols = ["task_id","code","name","type","startDate","endDate","status"]
select_cols = ["taskroleId","task.taskId"]
rename_cols = ["taskroleId","taskId"]
projects = projects[select_cols].copy()
projects.columns = rename_cols
#projects['dirName'] = projects['dirName'].fillna(projects['dirName2']).astype(str)

In [8]:
#projects

In [9]:
# Groupe data on taskId to get number of people working on a project
r_count = projects.groupby(by=["taskId"]).count()["taskroleId"].reset_index()
r_count.rename(columns={"taskroleId":"resources"}, inplace=True)
r_count
#projects.columns[15:]

,taskId,resources
0,2,1
1,3,1
2,4,2
3,8,1
4,9,1
5,10,1


In [10]:
#task
#r_count
df = pd.merge(left=task, right=r_count, how='left')
df["resources"] = df.resources.fillna(0).astype("int")
df["startDate"] = df.startDate.astype("str")
df["endDate"] = df.endDate.astype("str")

In [11]:
#df.to_dict(orient="records")